In [ ]:
from Helper.onnx_konv import *
import json
import os


In [ ]:
best_checkpoints_json = "FINAL_DATEN/best_checkpoints_Mapillary_small.json"
calibration_data = "Mapillary_Vistas/training/images"
output_dir = "onnx_models_mapillary_small"
num_classes = 124
calibration_cache_file = "Kalibr_Cache/cache_file.h5"
calibration_batch_size = 5  # Hier definierst du, wie viele Bilder pro Batch geladen werden sollen

with open(best_checkpoints_json, "r") as f:
    best_checkpoints = json.load(f)

model_names = sorted(best_checkpoints.keys())
print("Gefundene Modelle im angegebenen Ordnerpfad:")
for idx, name in enumerate(model_names, start=1):
    print(f"[{idx}] {name}")

conversion_mapping = {
    1: ["fp32"],
    2: ["fp32", "fp16", "fp8"],
    3: ["fp32", "fp16", "fp8", "int8_dynamic"],
    4: ["fp32", "fp16", "fp8", "int8_calibrated"],
    5: ["fp32", "fp16", "fp8", "int8_dynamic", "int8_calibrated"]
}
print("\nUmrechnungscodes:")
for code, convs in conversion_mapping.items():
    print(f"  {code}: {', '.join(convs)}")

In [ ]:
# Steuerung der Ausführung:
# run_all = True: Alle Modelle in allen Varianten (Code 5) werden konvertiert.
# Alternativ: Mit selected_model_indices und selected_conversion_codes gezielt auswählen.
run_all = False
selected_model_indices = [1, 2, 3, 4]      # Beispiel: 1. und 3. Modell
selected_conversion_codes = [2, 2, 2, 2]     # Beispiel: Modell 1 mit Code 2 (FP32+FP16), Modell 3 mit Code 4 (FP32+INT8 calibrated)


In [ ]:
# %% Cell 3B: Funktionsdefinitionen (kollabierbar)
def convert_and_test_model(checkpoint_path, model_name, conversion_types, calibration_data_path, num_classes):
    """
    Führt für das angegebene Modell (über den Checkpoint) die gewünschten Konvertierungen durch
    und testet anschließend alle erfolgreich konvertierten Modelle.
    
    :param checkpoint_path: Pfad zum Checkpoint (z. B. eine pickle-Datei)
    :param model_name: Name des Modells
    :param conversion_types: Liste der gewünschten Konvertierungen (z. B. ["fp32", "fp16", "int8_dynamic", "int8_calibrated"])
    :param calibration_data_path: Pfad zu den Kalibrierungsbildern (wichtig für INT8)
    :param num_classes: Anzahl der Klassen, die im Modell verwendet werden sollen
    """
    print(f"\n=== Starte Konvertierung für Modell: {model_name} ===")
    
    # Erstelle einen Converter (MapillaryONNXModelConverter nutzt hier den HDF5-Loader, falls calibration_cache_file existiert)
    converter = MapillaryONNXModelConverter(
        checkpoint_path=checkpoint_path,
        model_name=model_name,
        width=2048,  # ALtlast, hat kein Einfluss auf irgendwas, kann ignoriert werden
        height=1024,    # ALtlast, hat kein Einfluss auf irgendwas, kann ignoriert werden
        input_shape=(1, 3, 520, 520),
        num_classes=num_classes,
        dynamic_batch=True,
        opset_version=13,
        output_dir=output_dir,
        skip_local_load=True,
        calibration_cache_file=calibration_cache_file  # Hier wird der Cache-Pfad übergeben
    )
    
    # Modell laden
    try:
        converter.load_model()
    except Exception as e:
        print(f"[ERROR] Laden des Modells {model_name} fehlgeschlagen: {e}")
        return
    
    converted_paths = {}
    
    # FP32-Export
    if "fp32" in conversion_types or any(conv in conversion_types for conv in ["fp16", "int8_dynamic", "int8_calibrated"]):
        try:
            fp32_path = converter.export_fp32()
            converted_paths["fp32"] = fp32_path
        except Exception as e:
            print(f"[ERROR] FP32-Export von {model_name} fehlgeschlagen: {e}")
            return

    # FP16-Konvertierung
    if "fp16" in conversion_types:
        if HAS_FLOAT16_CONVERTER:
            try:
                fp16_path = converter.convert_fp16(fp32_path)
                converted_paths["fp16"] = fp16_path
            except Exception as e:
                print(f"[WARN] FP16-Konvertierung von {model_name} fehlgeschlagen: {e}")
        else:
            print("[WARN] FP16-Konvertierung nicht verfügbar (float16_converter fehlt).")
            
    # FP8-Konvertierung (analog zu FP16)
    if "fp8" in conversion_types:
        try:
            fp8_path = converter.convert_fp8(fp32_path)
            converted_paths["fp8"] = fp8_path
        except Exception as e:
            print(f"[WARN] FP8-Konvertierung von {model_name} fehlgeschlagen: {e}")

    
    # Dynamische INT8-Quantisierung
    if "int8_dynamic" in conversion_types:
        try:
            int8_dyn_path = converter.quantize_int8_dynamic(fp32_path)
            converted_paths["int8_dynamic"] = int8_dyn_path
        except Exception as e:
            print(f"[ERROR] Dynamische INT8-Quantisierung von {model_name} fehlgeschlagen: {e}")
    
    # Kalibrierte INT8-Quantisierung (nutzt den HDF5-Loader, falls Cache vorhanden, ansonsten den Lazy-Loader)
    if "int8_calibrated" in conversion_types:
        try:
            int8_cal_path = converter.quantize_int8_calibrated(
                fp32_onnx_path=fp32_path,
                calibration_data_path=calibration_data_path,
                input_tensor_name="input",
                max_samples=200
            )
            converted_paths["int8_calibrated"] = int8_cal_path
        except Exception as e:
            print(f"[ERROR] Kalibrierte INT8-Quantisierung von {model_name} fehlgeschlagen: {e}")
    
    # Teste alle erfolgreich konvertierten Modelle
    print(f"\n>>> Teste konvertierte Modelle für {model_name}:")
    for conv, path in converted_paths.items():
        try:
            print(f"{conv.upper()} Modell:")
            converter.test_model(path)
        except Exception as e:
            print(f"[ERROR] Testen des {conv.upper()} Modells von {model_name} fehlgeschlagen: {e}")
    
    print(f"=== Konvertierung für {model_name} abgeschlossen ===\n")


In [ ]:
# %% Cell 3C: Ausführung & Ausgabe
if run_all:
    for model_name in model_names:
        checkpoint_path = best_checkpoints[model_name]
        print(f"\n[RUN_ALL] Konvertiere Modell '{model_name}' mit allen Varianten...")
        convert_and_test_model(checkpoint_path, model_name, conversion_mapping[5], calibration_data, num_classes)
else:
    if len(selected_model_indices) != len(selected_conversion_codes):
        print("[ERROR] Die Listen selected_model_indices und selected_conversion_codes müssen gleich lang sein!")
    else:
        for idx, conv_code in zip(selected_model_indices, selected_conversion_codes):
            model_idx = idx - 1
            if model_idx < 0 or model_idx >= len(model_names):
                print(f"[ERROR] Ungültige Modellnummer: {idx}")
                continue
            model_name = model_names[model_idx]
            checkpoint_path = best_checkpoints[model_name]
            conversion_types = conversion_mapping.get(conv_code)
            if conversion_types is None:
                print(f"[ERROR] Ungültiger Umrechnungscode: {conv_code} für Modell '{model_name}'")
                continue
            print(f"\n[SELECTED] Konvertiere Modell '{model_name}' mit Umrechnungscode {conv_code} ({', '.join(conversion_types)})...")
            convert_and_test_model(checkpoint_path, model_name, conversion_types, calibration_data, num_classes)
